In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import io
import zipfile
import re
from datetime import datetime
import time
from urllib.parse import urljoin


In [3]:
# !pip install xlrd

In [6]:
!pip show xlrd

Name: xlrd
Version: 2.0.1
Summary: Library for developers to extract data from Microsoft Excel (tm) .xls spreadsheet files
Home-page: http://www.python-excel.org/
Author: Chris Withers
Author-email: chris@withers.org
License: BSD
Location: C:\Users\barid\anaconda3\Lib\site-packages
Requires: 
Required-by: 


In [7]:
import sys
print(sys.executable)

C:\Users\barid\anaconda3\envs\fuzzy_match\python.exe


In [8]:
! C:\Users\barid\anaconda3\envs\fuzzy_match\python.exe -m pip install xlrd --upgrade


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\barid\anaconda3\envs\fuzzy_match\python.exe -m pip install --upgrade pip


In [9]:
try:
    import xlrd
    print("xlrd is successfully imported.")
except ImportError:
    print("xlrd is NOT installed or accessible in the current kernel.")

xlrd is successfully imported.


In [28]:
def extract_municipal_advisor_data(url, project_nm, ver, email_id, start_year=2015, start_month=5):
    """
    Downloads and extracts municipal advisor data (Name, File Number, CIK, Year, Month)
    from SEC zipped and standalone .xlsx files, handling .xls formats.

    Args:
        url (str): The base URL of the SEC website.
        project_nm (str): The name of your project.
        ver (str): The version of your project.
        email_id (str): Your email address.
        start_year (int): The year from which to start processing files.
        start_month (int): The month from which to start processing files (1-12).

    Returns:
        pandas.DataFrame: A DataFrame containing the extracted data.
    """
    all_data = []
    project_name = project_nm
    version = ver
    email = email_id

    user_agent = f"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 ({project_name}/{version} {email})"
    headers = {'User-Agent': user_agent}
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for bad status codes
    soup = BeautifulSoup(response.content, 'html.parser')

    download_links_relative = [a['href'] for a in soup.find_all('a', href=True) if 'municipal-advisor' in a['href'] and ('.zip' in a['href'] or '.xlsx' in a['href'])]
    download_links = [urljoin(url, link) for link in download_links_relative]
    print("Number of links obtained =", len(download_links))
    # print("Download links collected = ", download_links)

    ctr=0
    for link in download_links:
        ctr=ctr+1
        print(f"Processing iteration for link # {ctr}")
        try:
            file_name = link.split('/')[-1]
            year = None
            month = None

            # Enhanced regex to handle various zip file name formats
            zip_match = re.search(r'ma[-]?(\d{1,2})(\d{1,2})(\d{2})-?.*\.zip', file_name)
            if zip_match:
                month = int(zip_match.group(1))
                day = int(zip_match.group(2))
                year_short = int(zip_match.group(3))
                year = 2000 + year_short if year_short <= 99 else year_short

            xlsx_match = re.search(r'ma.*?(\d{4})-(\d{2}).*\.xlsx', file_name, re.IGNORECASE)
            if not year and xlsx_match:
                year = int(xlsx_match.group(1))
                month = int(xlsx_match.group(2))
            elif not year:
                xlsx_match_short_year_hyphenated = re.search(r'ma.*?(\d{2})-(\d{2})-(\d{2}).*\.xlsx', file_name, re.IGNORECASE)
                if xlsx_match_short_year_hyphenated:
                    month = int(xlsx_match_short_year_hyphenated.group(1))
                    day = int(xlsx_match_short_year_hyphenated.group(2))
                    year_short = int(xlsx_match_short_year_hyphenated.group(3))
                    year = 2000 + year_short if year_short <= 99 else year_short
                else:
                    xlsx_match_short_year_no_hyphen = re.search(r'ma.*?(\d{2})(\d{2})(\d{2}).*\.xlsx', file_name, re.IGNORECASE)
                    if xlsx_match_short_year_no_hyphen:
                        month = int(xlsx_match_short_year_no_hyphen.group(1))
                        day = int(xlsx_match_short_year_no_hyphen.group(2))
                        year_short = int(xlsx_match_short_year_no_hyphen.group(3))
                        year = 2000 + year_short if year_short <= 99 else year_short
                    else:
                        print(f"Warning: Could not consistently extract year and month from {file_name}. Skipping file.")
                        continue

            if year is not None and (year > start_year or (year == start_year and month >= start_month)):
                print(f"Processing: {file_name}, Year: {year}, Month: {month}")
                response = requests.get(link, headers=headers)
                response.raise_for_status()

                if file_name.endswith('.zip'):
                    try:
                        with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
                            data_files = [f for f in zf.namelist() if f.endswith(('.xlsx', '.csv', '.xls'))]
                            if data_files:
                                data_file = data_files[0]
                                print(f"  Found data file: {data_file} in {file_name}")
                                with zf.open(data_file) as data:
                                    df = read_data_file(data, file_name)
                                    if df is not None and len(df.columns) >= 3:
                                        extract_and_append_data(df, year, month, all_data, file_name, data_file)
                                    elif df is not None:
                                        print(f"  Warning: Less than 3 columns found in {data_file} of {file_name}")
                            else:
                                print(f"    Warning: No Excel/CSV/XLS file found in {file_name}")
                    except zipfile.BadZipFile:
                        print(f"Error: Could not open {file_name} as a zip file.")
                    except Exception as e:
                        print(f"An unexpected error occurred while processing zip file {file_name}: {e}")
                elif file_name.endswith('.xlsx'):
                    try:
                        df = pd.read_excel(io.BytesIO(response.content), header=2, engine='openpyxl')
                        print(f"  Successfully read as Excel (xlsx). Columns: {df.columns.tolist()}")
                        if len(df.columns) >= 3:
                            extract_and_append_data(df, year, month, all_data, file_name, file_name) # Use filename as data_file for standalone xlsx
                        else:
                            print(f"  Warning: Less than 3 columns found in {file_name}")
                    except ImportError:
                        print("  Warning: openpyxl not installed. Cannot read xlsx files.")
                    except Exception as e_xlsx:
                        try:
                            df = pd.read_excel(io.BytesIO(response.content), header=2, engine='xlrd')
                            print(f"  Successfully read as Excel (xls). Columns: {df.columns.tolist()}")
                            if len(df.columns) >= 3:
                                extract_and_append_data(df, year, month, all_data, file_name, file_name) # Use filename as data_file for standalone xlsx
                            else:
                                print(f"  Warning: Less than 3 columns found in {file_name}")
                        except ImportError:
                            print("  Warning: xlrd not installed. Cannot read xls files.")
                        except Exception as e_xls:
                            print(f"  Error reading {file_name} as standalone Excel: {e_xlsx}, {e_xls}")
                time.sleep(1)
        except requests.exceptions.RequestException as e:
            print(f"Error downloading {link}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred while processing {link}: {e}")

    final_df = pd.DataFrame(all_data)
    return final_df


In [30]:
def read_data_file(data, zip_file_name):
    """Tries to read the data file (xlsx, csv, xls) from the zip archive."""
    df = None
    try:
        df = pd.read_excel(data, header=2, engine='openpyxl')
        print(f"    Successfully read as Excel (xlsx).")
    except ImportError:
        print("    Warning: openpyxl not installed. Cannot read xlsx files.")
    except Exception as e_xlsx:
        try:
            data.seek(0)
            df = pd.read_csv(data, skiprows=2)
            print(f"    Successfully read as CSV.")
        except Exception as e_csv:
            try:
                data.seek(0)
                df = pd.read_excel(data, header=2, engine='xlrd')
                print(f"    Successfully read as Excel (xls).")
            except ImportError:
                print("    Warning: xlrd not installed. Cannot read xls files.")
            except Exception as e_xls:
                print(f"    Error reading data file as xls: {e_xls}")
                print(f"    Error reading data file as both Excel and CSV: {e_xlsx}, {e_csv}")
    return df



In [32]:
def extract_and_append_data(df, year, month, all_data, file_name, data_file_name):
    """Identifies columns and appends data to the all_data list (including 'Company' and 'File No')."""
    cik_col = None
    registrant_name_col = None
    file_number_col = None

    for col in df.columns:
        lower_col = col.strip().lower()
        if 'cik' in lower_col or 'central index key' in lower_col:
            cik_col = col
        if 'registrant name' in lower_col or 'firm name' in lower_col or 'municipal advisor' in lower_col or 'name of municipal advisor' in lower_col or 'company' in lower_col:
            registrant_name_col = col
        if 'file no' in lower_col or 'file number' in lower_col or 'sec file no' in lower_col or 'sec file number' in lower_col:
            file_number_col = col

    if registrant_name_col and file_number_col and cik_col:
        for index, row in df.iterrows():
            name = row.get(registrant_name_col)
            file_no = row.get(file_number_col)
            cik = str(int(float(row.get(cik_col)))) if pd.notna(row.get(cik_col)) else None
            if cik:
                all_data.append({
                    'Registrant Name': name,
                    'File Number': file_no,
                    'CIK': cik.zfill(10),
                    'Year': year,
                    'Month': month
                })
    else:
        print(f"    Warning: Could not identify all required columns in {data_file_name} of {file_name}. Columns found: {df.columns.tolist()}")

In [34]:
if __name__ == "__main__":
    sec_base_url = "https://www.sec.gov"  # Use the base URL
    relative_path = "/data-research/sec-markets-data/information-about-registered-municipal-advisors"
    full_url = urljoin(sec_base_url, relative_path)
    project_name = "get_cik"  # Replace with your project name
    version = "1.0"  # Replace with your project version
    email = "bmalakar3@gatech.edu"  # Replace with your email address
    # municipal_advisor_df = extract_municipal_advisor_data(full_url, project_name, version, email, start_year=2015, start_month=5)
    municipal_advisor_df = extract_municipal_advisor_data(full_url, project_name, version, email, start_year=2015, start_month=5)
    print("\nCompiled Municipal Advisor CIKs:")
    print(municipal_advisor_df)
    
    # save the DataFrame to a CSV file if needed
    # municipal_advisor_ciks_df.to_csv("municipal_advisor_ciks.csv", index=False)

Number of links obtained = 119
Processing iteration for link # 1
Processing: ma040125.xlsx, Year: 2025, Month: 4
  Successfully read as Excel (xlsx). Columns: ['Registrant Name', 'File Number', 'CIK']
Processing iteration for link # 2
Processing: ma030125.xlsx, Year: 2025, Month: 3
  Successfully read as Excel (xlsx). Columns: ['Registrant Name', 'File Number', 'CIK']
Processing iteration for link # 3
Processing: ma020125.xlsx, Year: 2025, Month: 2
  Successfully read as Excel (xlsx). Columns: ['Registrant Name', 'File Number', 'CIK']
Processing iteration for link # 4
Processing: ma010125.xlsx, Year: 2025, Month: 1
  Successfully read as Excel (xlsx). Columns: ['Registrant Name', 'File Number', 'CIK']
Processing iteration for link # 5
Processing: ma120124.xlsx, Year: 2024, Month: 12
  Successfully read as Excel (xlsx). Columns: ['Registrant Name', 'File Number', 'CIK']
Processing iteration for link # 6
Processing: ma110124.xlsx, Year: 2024, Month: 11
  Successfully read as Excel (xlsx)

In [38]:
print(municipal_advisor_df.columns)

Index(['Registrant Name', 'File Number', 'CIK', 'Year', 'Month'], dtype='object')


In [40]:
municipal_advisor_df.describe()


,Year,Month
count,64841.000000,64841.000000
mean,2019.386345,6.553045
std,2.890556,3.439048
min,2015.000000,1.000000
25%,2017.000000,4.000000
50%,2019.000000,7.000000
75%,2022.000000,10.000000
max,2025.000000,12.000000


In [42]:
municipal_advisor_df['CIK'].nunique()


920

In [44]:
# count the number of unique year-month combinations in the df
municipal_advisor_df['yearMth'] = municipal_advisor_df['Year'].astype(str)+ '-' + municipal_advisor_df['Month'].astype(str).str.zfill(2)
yr_mth_count = municipal_advisor_df.groupby('yearMth').size()
n_uniq_yrMth = len(yr_mth_count)

print(f"Num of unique year mth combis ={n_uniq_yrMth}")
print("\nBreakdown of counts per month:")
print(yr_mth_count)

Num of unique year mth combis =119

Breakdown of counts per month:
yearMth
2015-05    633
2015-06    638
2015-07    647
2015-08    653
2015-09    661
          ... 
2024-12    423
2025-01    427
2025-02    424
2025-03    422
2025-04    422
Length: 119, dtype: int64


In [46]:
uniq_mths_byYr = municipal_advisor_df.groupby('Year')['Month'].nunique()
uniq_mths_byYr

Year
2015     8
2016    12
2017    12
2018    12
2019    11
2020    12
2021    12
2022    12
2023    12
2024    12
2025     4
Name: Month, dtype: int64

In [ ]:
# The above looks okay for now. Note, SEC website does not have March 2019 file, so 11 in 2019 is correct.
# In 2015, we start from May when the Excel files were introduced; so 8 is correct.

In [48]:
# Extracted the pdf's of 2015 manually

file1 = pd.read_excel(r"C:\Users\barid\Documents\Papers\Muni_adv_PBF\ma_filings\ma031515.xlsx", engine='openpyxl')
file1['Year'] = 2015
file1['Month'] = 3
file1 = file1.rename(columns={'Company Name':'Registrant Name'})
file1

,Registrant Name,File No,CIK,Year,Month
0,1410 Partners LLC,867-01318,1620673,2015,3
1,A. Dashen & Associates,867-00672,1617911,2015,3
2,"A. M. Miller & Co., Inc.",867-00872,1618730,2015,3
3,A. M. Peche & Associates LLC,867-00111,1613201,2015,3
4,A.BRIDGE REALVEST SECURITIES CORPORATION,867-01291,1005399,2015,3
...,...,...,...,...,...
608,"YOUNG AMERICA CAPITAL, LLC",867-01657,1463911,2015,3
609,Yuba Group LLC,867-00105,1612952,2015,3
610,ZIONS FIRST NATIONAL BANK /MSD,867-00724,797595,2015,3
611,"Zions Public Finance, Inc.",867-01453,1628261,2015,3


In [50]:
file2 = pd.read_excel(r"C:\Users\barid\Documents\Papers\Muni_adv_PBF\ma_filings\ma040115.xlsx", engine='openpyxl')
file2['Year'] = 2015
file2['Month'] = 4
file2 = file2.rename(columns={'Company':'Registrant Name'})
file2

,Registrant Name,File No,CIK,Year,Month
0,1410 Partners LLC,867-01318,1620673,2015,4
1,A. Dashen & Associates,867-00672,1617911,2015,4
2,"A. M. Miller & Co., Inc.",867-00872,1618730,2015,4
3,A. M. Peche & Associates LLC,867-00111,1613201,2015,4
4,A.BRIDGE REALVEST SECURITIES CORPORATION,867-01291,1005399,2015,4
...,...,...,...,...,...
622,"YOUNG AMERICA CAPITAL, LLC",867-01657,1463911,2015,4
623,Yuba Group LLC,867-00105,1612952,2015,4
624,ZIONS FIRST NATIONAL BANK /MSD,867-00724,797595,2015,4
625,"Zions Public Finance, Inc.",867-01453,1628261,2015,4


In [52]:
muni_adv2 = pd.concat([municipal_advisor_df, file1, file2], ignore_index=True)
muni_adv2

,Registrant Name,File Number,CIK,Year,Month,yearMth,File No
0,&PARTNERS,867-01682,0000107136,2025,4,2025-04,NaN
1,"30 Three Sixty Public Finance, Inc.",867-02350,0001733578,2025,4,2025-04,NaN
2,A. M. Peche & Associates LLC,867-00111,0001613201,2025,4,2025-04,NaN
3,A.BRIDGE REALVEST SECURITIES CORPORATION,867-01291,0001005399,2025,4,2025-04,NaN
4,A&C Galvan Group LLC,867-02679,0002013292,2025,4,2025-04,NaN
...,...,...,...,...,...,...,...
66076,"YOUNG AMERICA CAPITAL, LLC",NaN,1463911,2015,4,NaN,867-01657
66077,Yuba Group LLC,NaN,1612952,2015,4,NaN,867-00105
66078,ZIONS FIRST NATIONAL BANK /MSD,NaN,797595,2015,4,NaN,867-00724
66079,"Zions Public Finance, Inc.",NaN,1628261,2015,4,NaN,867-01453


In [54]:
uniq_mths_byYr = muni_adv2.groupby('Year')['Month'].nunique()
uniq_mths_byYr

Year
2015    10
2016    12
2017    12
2018    12
2019    11
2020    12
2021    12
2022    12
2023    12
2024    12
2025     4
Name: Month, dtype: int64

In [56]:
# export to csv
muni_adv2.to_csv(r"C:\Users\barid\Documents\Papers\Muni_adv_PBF\ma_filings\sec_cik_ma.csv", index=False)